In [1]:
import (
	"encoding/json"
    . "github.com/FactomProject/ptnet-eventstore/contract"
	"github.com/FactomProject/ptnet-eventstore/finite"
	. "github.com/FactomProject/ptnet-eventstore/identity"
	. "github.com/FactomProject/ptnet-eventstore/ptnet"
	"github.com/FactomProject/ptnet-eventstore/x"
)

In [2]:
// the state machine definition composes the rules of the contract
// machine must be designed to always end in a halting state
// this protocol requires the machine to be halted before tokens are released
//
// NOTE:
// this type of state machine classified as a https://en.wikipedia.org/wiki/Vector_addition_system
// and can also be represented graphically as a https://en.wikipedia.org/wiki/Petri_net
m := Machine{
    // define the inital state - a nice convention follow
    // is to set each variable to the max possible value
    // that can be achieved during valid contract execution
    // in this contract no value ever exceeds 1
	//                   00 01 02 10 11 12 20 21 22 O  X $O $X $DEP
	Initial: StateVector{1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1},

	Transitions: map[string]Transition{
        // each transition defines its effect 
        // on every offset 'variable' declared in the state vector
        //
        // NOTE: this is mathematical vector addition at work
        // each vector is 'executed' by adding it to the preceding state
		//                00 01 02 10 11 12 20 21 22  O  X $O $X $DEP
        // by convention a contract must have a BEGIN/"EXEC" transaction
        // this action is executed when the contract is first offered
		BEGIN: Transition{0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1, -1, -1},
        // moves for player X
		"X00": Transition{-1, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, 0, 0, 0},
		"X01": Transition{0, -1, 0, 0, 0, 0, 0, 0, 0, 1, -1, 0, 0, 0},
		"X02": Transition{0, 0, -1, 0, 0, 0, 0, 0, 0, 1, -1, 0, 0, 0},
		"X10": Transition{0, 0, 0, -1, 0, 0, 0, 0, 0, 1, -1, 0, 0, 0},
		"X11": Transition{0, 0, 0, 0, -1, 0, 0, 0, 0, 1, -1, 0, 0, 0},
		"X12": Transition{0, 0, 0, 0, 0, -1, 0, 0, 0, 1, -1, 0, 0, 0},
		"X20": Transition{0, 0, 0, 0, 0, 0, -1, 0, 0, 1, -1, 0, 0, 0},
		"X21": Transition{0, 0, 0, 0, 0, 0, 0, -1, 0, 1, -1, 0, 0, 0},
		"X22": Transition{0, 0, 0, 0, 0, 0, 0, 0, -1, 1, -1, 0, 0, 0},
        // moves for player O
		"O00": Transition{-1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0},
		"O01": Transition{0, -1, 0, 0, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0},
		"O02": Transition{0, 0, -1, 0, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0},
		"O10": Transition{0, 0, 0, -1, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0},
		"O11": Transition{0, 0, 0, 0, -1, 0, 0, 0, 0, -1, 1, 0, 0, 0},
		"O12": Transition{0, 0, 0, 0, 0, -1, 0, 0, 0, -1, 1, 0, 0, 0},
		"O20": Transition{0, 0, 0, 0, 0, 0, -1, 0, 0, -1, 1, 0, 0, 0},
		"O21": Transition{0, 0, 0, 0, 0, 0, 0, -1, 0, -1, 1, 0, 0, 0},
		"O22": Transition{0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 1, 0, 0, 0},

        // moves that halt game and unlock tokens for winner
        // or refund to the depositor
		"WINX": Transition{0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 1, 0},
		"WINO": Transition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1, 0, 0},
		"ENDX": Transition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 1},
		"ENDO": Transition{0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 1},
	},
}

In [3]:
// contract declaration adds more rules in addition to the state machine
gameContract := Declaration{
    // inputs are similar to most cryptocurrency rules
    // and represent a type of 'pay-to-script' functionality
    // tokens are lock away in the contract until the blockheight is exceeded (invalid/fault)
    // or the contract halts (valid completion)
	Inputs: []AddressAmountMap{
		AddressAmountMap{Address[DEPOSITOR], 1}, // deposit 1 token
	},
    // outputs are referenced by Guards and Conditions
    // and are used as an Access Control List (ACL)
    // to define roles of users that are allowed to interact
    // with the contract in addition to defining which users
    // can withdraw tokens from a completed/halted contract
	Outputs: []AddressAmountMap{ // array of possible redeemers
		AddressAmountMap{Address[DEPOSITOR], 1},
		AddressAmountMap{Address[PLAYERX], 1},
		AddressAmountMap{Address[PLAYERO], 1},
	},
	BlockHeight: 60221409,      // deadline to achieve halting state
	Salt:        "|RANDOM|",    // random salt
	ContractID:  "|OctoeContractID|",// unique ID for this contract instance
	Schema:      "octoe-v1",    // versioned contract schema
	State:       m.Initial,     // state machine initial state
	Actions:     m.Transitions, // state machine defined transitions
	Guards: []Condition{ // guards restrict state machine actions
		//       00 01 02 10 11 12 20 21 22  O  X $O $X $DEP
		Condition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0}, // Admin/Depositor
		Condition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0},// PlayerX turn
		Condition{0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0},// PlayerO turn
	},
	Conditions: []Condition{ // redeem conditions to unlock input tokens
		//       00 01 02 10 11 12 20 21 22  O  X $O $X $DEP
		Condition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1},// no win/refund deposit
		Condition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0},// X win
		Condition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0},// O win 
	},
}

In [4]:
// use declaration to construct an offer
// this is a data wrapper that is used to isolate transaction ops from declaration ops
// NOTE: mostly this is just an artifact of this implementation not especially meaningful
offer := finite.Offer{
    ChainID: CHAIN_ID,
	Declaration: gameContract,
}

In [5]:
offer.String()


ChainID: |ChainID|
Inputs: 
	Address: dd808299f72f362e3cab3b8d1955505133ed1d1c72db295108f8896f767aeb7b Amount: 1 
Outputs: 
	Address: dd808299f72f362e3cab3b8d1955505133ed1d1c72db295108f8896f767aeb7b Amount: 1 
	Address: 947b35b8e4dad5957e24ed0bf02903c6390863753c753106a778111000cefa04 Amount: 1 
	Address: 1c6334d78fd8a6879c7143d2cf16ad7559d5e37c1587c2025fa44b41be597db7 Amount: 1 
BlockHeight: 60221409
Salt: |RANDOM|
ContractID: |OctoeContractID|
Schema: octoe-v1
State: [1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Actions: 
	ENDO: [0 0 0 0 0 0 0 0 0 -1 0 0 0 1]
	ENDX: [0 0 0 0 0 0 0 0 0 0 -1 0 0 1]
	EXEC: [0 0 0 0 0 0 0 0 0 -1 0 -1 -1 -1]
	O00: [-1 0 0 0 0 0 0 0 0 -1 1 0 0 0]
	O01: [0 -1 0 0 0 0 0 0 0 -1 1 0 0 0]
	O02: [0 0 -1 0 0 0 0 0 0 -1 1 0 0 0]
	O10: [0 0 0 -1 0 0 0 0 0 -1 1 0 0 0]
	O11: [0 0 0 0 -1 0 0 0 0 -1 1 0 0 0]
	O12: [0 0 0 0 0 -1 0 0 0 -1 1 0 0 0]
	O20: [0 0 0 0 0 0 -1 0 0 -1 1 0 0 0]
	O21: [0 0 0 0 0 0 0 -1 0 -1 1 0 0 0]
	O22: [0 0 0 0 0 0 0 0 -1 -1 1 0 0 0]
	WINO: [0 0 0 0 0 0 0 0 0 0

In [6]:
// check to see if contract has been recorded yet (it hasn't)
Exists(gameContract.Schema, gameContract.ContractID)

false

In [7]:
// to begin contract execution an - offer transaction is made
// by convention the initial state should be set to the upper bound
// for all the variables used by the contract
// in this contract no variable is ever > 1
//
// NOTE by design - the state machine reports any output values < 0 as invalid
// see: https://en.wikipedia.org/wiki/Vector_addition_system for more details
txn := finite.OfferTransaction(offer, Private[DEPOSITOR])
txn.String() // notice the event below contains the json-encoded contract declaration


Timestamp:   1542727538946277646
Schema:      octoe-v1
Action:      EXEC
Oid:         |OctoeContractID|
Value:       1
InputState:  [1 1 1 1 1 1 1 1 1 1 1 1 1 1]
OutputState: [1 1 1 1 1 1 1 1 1 0 1 0 0 0]
Payload:
	1f8b08000000000000ffacd4cf8fa2301407f0ffe55da7665b708ddb640f1330ea64069455d61fe1808551560557caac338efffb0644844a3d4df060da0fafedfba61c210877098f81ce8fe07adede8f63a0a04e356dfbcade8ca0d9fefb91185ddb1e8e4703deb30eefcafbc6d206c1d3429db55a6afc1310b8db280939507272104409ffca8aa85c65d38b49628e5c32b31f9e4cfcdcd6d6fae1cfba3f55c76d6fd4e9eed4a5d579d4cd87e5f04e955ef70593e6b7e92ef2191b0f7e7d1c5ceed9a1dd54ffd9ab765f9bb4ace7e1e2fbec75c16e4eb7d8446cbdf283e58a036d6145214dfc0341ec6e3850f8b41e0ddd7cf904042c0af9de653cf0d27193f1c8d7f2a1be9e8298adfcad0b14a274aef146d231ee721fe89c20e9e32048ab46610cf4081d433781ce31129f06c9ffa42f740c7d52ab0a97a9494793d76a90f3cf4160620c745eac5081f960a64856edc6094ac9d7145c55115cecace204454afb2f394129955316aeaa142cf422778222371dbb74abac949abe9e1b7a55bffb467d94d76a67551f

In [8]:
// check to see if our contract has been recorded
Exists(gameContract.Schema, gameContract.ContractID)

true

In [9]:
// show that contract is still open
IsHalted(gameContract)

false

In [10]:
// show that tokens are locked
// even original depositor cannot withdraw tokens
CanRedeem(gameContract, Public[DEPOSITOR])

false

In [11]:
// Helper function to make it easier to compose transactions in this example
func commit(action string, key PrivateKey, payload []byte) (finite.Transaction, error) {
	pub := PublicKey{}
	copy(pub[:], x.PrivateKeyToPub(key[:])) // get public key
	return finite.ExecuteTransaction(finite.Execution{
		Command: Command{
			ChainID:    CHAIN_ID,// which chain to add entry
			ContractID: "|OctoeContractID|",
			Schema:     OctoeV1, // state machine version
			Action:     action,  // state machine action
			Amount:     1,       // triggers input action 'n' times amount
			Payload:    payload, // arbitrary data optionally included
			Pubkey:     pub,     // must match key in contract declaration
		},
	}, key)
}

In [12]:
// an event with an arbitrary payload attached
// on-chain it is expected that this payload is used to attach
// additional supporting data or evidence that 
// an action is being executed according to the intent of the contract
payload, _ := json.Marshal([]string{"hello", "world"})
event1, _ := commit("X11", Private[PLAYERX], payload)
event1.String()


Timestamp:   1542727538949096031
Schema:      octoe-v1
Action:      X11
Oid:         |OctoeContractID|
Value:       1
InputState:  [1 1 1 1 1 1 1 1 1 0 1 0 0 0]
OutputState: [1 1 1 1 0 1 1 1 1 1 0 0 0 0]
Payload:
	1f8b08000000000000ff8a56ca48cdc9c957d2512acf2fca49518a05000000ffff010000ffff18681e1211000000
digest:
	f64228e127dfd6a7797851cc652a5db3d38427e836ce89a5fd926c4536572c81
pubkeys: 
	f09b40a1ceb3cd37f7a3584bbcccde13ad1d8313254f8d6fb3870786264179bf
signatures: 
	2f8f0c94449b38121780e8e77789ea20ef0ad62576f782c6f3ea730cd3ef52895aef9f303feba2b0f6c4a60adeaf7afc9759a5b5f0a132618e0b1ddd07b35e0a


In [13]:
// show an invalid move
// according to this state machine players must take turns
_, err := commit("X00", Private[PLAYERX], nil)
err

invalid output: -1 offset: 10

In [14]:
// another invalid move - here we try to sign with an incorrect key
// contract makes players sign each event
_, err := commit("O01", Private[PLAYERX], nil)
err

failed guard condition

In [15]:
// valid
event2, _ := commit("O01", Private[PLAYERO], nil)
event2.String()


Timestamp:   1542727538951758593
Schema:      octoe-v1
Action:      O01
Oid:         |OctoeContractID|
Value:       1
InputState:  [1 1 1 1 0 1 1 1 1 1 0 0 0 0]
OutputState: [1 0 1 1 0 1 1 1 1 0 1 0 0 0]
Payload:
	
digest:
	db403f7d310b2c2d84a2c9082c45d3742754664555d6250abb57ef0dc03a8d39
pubkeys: 
	843ec312a748b8c42989281ca090d0c659f6a03e8346edf7d4d512a8b76a76ef
signatures: 
	8daadb2c0ba631ec17b1acdb7f9d0877f8f10fe1735967e58c09bf3a7477f834cc6b309d05ebf8bbadee90b5f224207318d08a4e375b4b3a959b83ff84a49800


In [16]:
// invalid - state machine ensures move is used only once
// move X11 is already taken as seen above
_, err := commit("X11", Private[PLAYERX], nil)
err

invalid output: -1 offset: 4

In [17]:
// valid
event3, _ := commit("X00", Private[PLAYERX], nil)
event3.String()


Timestamp:   1542727538953140133
Schema:      octoe-v1
Action:      X00
Oid:         |OctoeContractID|
Value:       1
InputState:  [1 0 1 1 0 1 1 1 1 0 1 0 0 0]
OutputState: [0 0 1 1 0 1 1 1 1 1 0 0 0 0]
Payload:
	
digest:
	1e6cad21b5b3ce95f1c27c61ecce9cabd517c219917d62382c6c03250baf3afb
pubkeys: 
	f09b40a1ceb3cd37f7a3584bbcccde13ad1d8313254f8d6fb3870786264179bf
signatures: 
	e7112e843681744e80155685bc15045ed174dd250ca25a4061fec5512865044c1372697499f334c35e7b2f6cf2a718c8011734c6c2974bbbdee32f29245a4206


In [18]:
// valid
event4, _ := commit("O02", Private[PLAYERO], nil)
event4.String()


Timestamp:   1542727538953886997
Schema:      octoe-v1
Action:      O02
Oid:         |OctoeContractID|
Value:       1
InputState:  [0 0 1 1 0 1 1 1 1 1 0 0 0 0]
OutputState: [0 0 0 1 0 1 1 1 1 0 1 0 0 0]
Payload:
	
digest:
	06f7230c5e213bb97b4ab50c3bb1322c826b6360bec27545b6971a04e146b0b0
pubkeys: 
	843ec312a748b8c42989281ca090d0c659f6a03e8346edf7d4d512a8b76a76ef
signatures: 
	aa018696d63d02d8ab47d8358428bbf405cf0422a18fa95114c762982a599722fc4a63a59ba1a49acdab1f1d8d17302e64e9f9bf0941352534544949449c8304


In [19]:
// valid
event5, _ := commit("X22", Private[PLAYERX], nil)
event5.String()


Timestamp:   1542727538954690941
Schema:      octoe-v1
Action:      X22
Oid:         |OctoeContractID|
Value:       1
InputState:  [0 0 0 1 0 1 1 1 1 0 1 0 0 0]
OutputState: [0 0 0 1 0 1 1 1 0 1 0 0 0 0]
Payload:
	
digest:
	b19ee8963f8a563fa653202e1891d685fbde4356e3fc3ff5f29583d843150824
pubkeys: 
	f09b40a1ceb3cd37f7a3584bbcccde13ad1d8313254f8d6fb3870786264179bf
signatures: 
	64e100946c25d4dd23c44d58a6641099384da46988b99f24c37304f00ec11cc45a968661fa972aa7c986b61d9a22cbe8c81668d7a2278ea03c6445bf1b5ff301


In [20]:
// game is over
// contract depositor confirms the winner
// this halts the state machine thereby unlocking the deposited tokens
event6, _ := commit("WINX", Private[DEPOSITOR], nil)
event6.String()


Timestamp:   1542727538955406491
Schema:      octoe-v1
Action:      WINX
Oid:         |OctoeContractID|
Value:       1
InputState:  [0 0 0 1 0 1 1 1 0 1 0 0 0 0]
OutputState: [0 0 0 1 0 1 1 1 0 0 0 0 1 0]
Payload:
	
digest:
	4b8c81ef904b8d003486939d22f8f359e9decfa104efd3d8bbe0388f11cf1135
pubkeys: 
	824e2a348148276ccc0013c12bd43eb07b4f486decef781cbff6ecb72105cdfc
signatures: 
	0bfc3dcaa077f639d9e8340941a2fc5fc65d3ba727343085d05b884568d152d37dde9ceea5644ed4ae4c428f6d50aa29a07d6313c7b27d7271db6d87308ac703


In [21]:
// contract is only redeemable after halting
IsHalted(gameContract)

true

In [22]:
// only the winner can redeem the deposit
CanRedeem(gameContract, Public[PLAYERX])

true

In [23]:
// loser has no claim
CanRedeem(gameContract, Public[PLAYERO])

false